# FRACKTAL: Symbolic Tree Visualization

## Interactive Visualization of Recursive Symbolic Ontology Trees

This notebook demonstrates how FRACKTAL creates and visualizes its recursive symbolic ontology trees.

In [ ]:
# Import FRACKTAL
import sys
sys.path.append('..')

from fracktal import RecursiveFRSOE
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

print("✅ FRACKTAL imported successfully!")

In [ ]:
# Create sample data
sample_data = {
    'repetitive': 'Hello World! Hello World! Hello World!',
    'structured': 'Name: John, Age: 30, City: NYC, Name: Jane, Age: 25, City: LA',
    'code': 'def function(): return True; def function(): return True'
}

print("📊 Sample data created:")
for name, data in sample_data.items():
    print(f"   {name}: {len(data)} characters")

In [ ]:
# Initialize FRACKTAL and process data
fracktal = RecursiveFRSOE(hash_depth=3, symbol_range=5000)

trees = {}
for name, data in sample_data.items():
    print(f"\n🌳 Processing {name}...")
    compressed_result = fracktal.compress(data)
    symbolic_tree = compressed_result['codex_map']['symbolic_tree']
    trees[name] = symbolic_tree
    
    print(f"   📏 Original: {len(data)} characters")
    print(f"   🌿 Tree nodes: {symbolic_tree.node_count}")
    print(f"   📊 Max depth: {symbolic_tree.max_depth}")
    print(f"   🔗 Root nodes: {len(symbolic_tree.root_nodes)}")

In [ ]:
# Create tree visualizations
def visualize_tree(tree, title):
    G = nx.DiGraph()
    
    # Add nodes and edges
    for symbol, data in tree.nodes.items():
        G.add_node(symbol, **data)
        if data.get('link'):
            G.add_edge(data['link'], symbol)
    
    # Calculate layout
    pos = nx.spring_layout(G, k=2, iterations=50)
    
    # Create visualization
    plt.figure(figsize=(10, 8))
    nx.draw(G, pos, with_labels=True, node_color='lightblue', 
            node_size=1000, font_size=8, font_weight='bold',
            arrows=True, edge_color='gray', arrowsize=20)
    plt.title(title)
    plt.show()

print("🎨 Creating tree visualizations...")
for name, tree in trees.items():
    print(f"\n🌳 Visualizing {name} tree...")
    visualize_tree(tree, f"{name.title()} - Symbolic Tree")
    
    # Show tree statistics
    print(f"   📊 Tree Statistics:")
    print(f"      Nodes: {tree.node_count}")
    print(f"      Max Depth: {tree.max_depth}")
    print(f"      Root Nodes: {len(tree.root_nodes)}")

## Key Insights

🌳 **Tree Structure Patterns**:
- **Repetitive data** creates shallow, wide trees with many root nodes
- **Structured data** creates deeper trees with clear hierarchical patterns
- **Code-like data** creates complex trees with multiple branches

🔍 **Symbolic Intelligence**:
- FRACKTAL automatically identifies meaningful patterns
- Tree depth reflects data complexity
- Root nodes represent unique semantic units

**FRACKTAL**: Where data becomes visual intelligence! 🌳✨